# Import Required Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import joblib

# Data Preparation

In [3]:
# Load the dataset
corrupted_dataset = pd.read_csv("parking_citations.corrupted.csv")
corrupted_dataset

/home/nachiketh/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,1103341116,2015-12-21T00:00:00,1251.0,NaN,NaN,CA,200304.0,NaN,NaN,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,9.999900e+04,9.999900e+04
1,1103700150,2015-12-21T00:00:00,1435.0,NaN,NaN,CA,201512.0,NaN,NaN,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,9.999900e+04,9.999900e+04
2,1104803000,2015-12-21T00:00:00,2055.0,NaN,NaN,CA,201503.0,NaN,NaN,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6.439998e+06,1.802686e+06
3,1104820732,2015-12-26T00:00:00,1515.0,NaN,NaN,CA,NaN,NaN,NaN,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6.440041e+06,1.802686e+06
4,1105461453,2015-09-15T00:00:00,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,9.999900e+04,9.999900e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8726009,4347602394,2019-01-10T00:00:00,1245.0,VN686B,NaN,CA,201905.0,NaN,NaN,PA,WT,14301 DELANO ST,00300,53.0,88.13B+,METER EXP.,63.0,6.427279e+06,1.889149e+06
8726010,4347602405,2019-01-10T00:00:00,1350.0,VN316,NaN,CA,201905.0,NaN,HYUN,PA,SL,14500 FRIAR ST,00300,53.0,88.13B+,METER EXP.,63.0,6.425963e+06,1.890254e+06
8726011,4347602416,2019-01-10T00:00:00,1354.0,VN914,NaN,CA,201912.0,NaN,NISS,PA,GY,14400 FRIAR ST,00300,53.0,88.13B+,METER EXP.,63.0,6.426623e+06,1.890252e+06
8726012,4347602420,2019-01-10T00:00:00,1411.0,VN725,NaN,CA,201903.0,NaN,NaN,PA,GY,6300 SYLMAR AV,00300,53.0,88.13B+,METER EXP.,63.0,6.426622e+06,1.889882e+06


In [4]:
#Check for null values
corrupted_dataset.isna().sum()

Ticket number                  0
Issue Date                   536
Issue time                  2583
Meter Id                 6456512
Marked Time              8435415
RP State Plate               765
Plate Expiry Date         794827
VIN                      8709705
Make                     4368470
Body Style                  8890
Color                       4115
Location                     854
Route                      65354
Agency                       545
Violation code                 0
Violation Description        872
Fine amount                 6507
Latitude                       3
Longitude                      3
dtype: int64

In [5]:
# Dataset removing all nan values in make - half
all_make_without_nan_dataset = corrupted_dataset[corrupted_dataset['Make'].notnull()]

In [6]:
# Dataset with make values as null - (Test dataset)
test = corrupted_dataset[corrupted_dataset['Make'].isnull()]

In [7]:
# Top 25 Make Values
top_25_make = all_make_without_nan_dataset.groupby('Make').agg('count').sort_values(['Ticket number'], ascending=False)[:25]

In [8]:
top_25_make_list = top_25_make.index.tolist()

In [9]:
top_25_make_list

['TOYT',
 'HOND',
 'FORD',
 'NISS',
 'CHEV',
 'BMW',
 'MERZ',
 'VOLK',
 'HYUN',
 'DODG',
 'LEXS',
 'KIA',
 'JEEP',
 'AUDI',
 'MAZD',
 'OTHR',
 'GMC',
 'CHRY',
 'INFI',
 'ACUR',
 'SUBA',
 'VOLV',
 'TOYO',
 'MITS',
 'CADI']

In [10]:
# Filtering all the rows which are not part of top 25 make list
boolean_series = all_make_without_nan_dataset.Make.isin(top_25_make_list)
filtered_series = all_make_without_nan_dataset[boolean_series]

In [11]:
train = filtered_series
train

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
4,1105461453,2015-09-15T00:00:00,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,9.999900e+04,9.999900e+04
6,1106500452,2015-12-17T00:00:00,1710.0,NaN,NaN,CA,201605.0,NaN,MAZD,PA,BL,SUNSET/ALVARADO,00217,1.0,8070,PARK IN GRID LOCK ZN,163.0,9.999900e+04,9.999900e+04
10,1106506424,2015-12-22T00:00:00,1100.0,NaN,NaN,CA,201511.0,NaN,FORD,TR,WH,1159 HUNTLEY DR,2A75,1.0,8069AA,NO STOP/STAND AM,93.0,9.999900e+04,9.999900e+04
11,1106506435,2015-12-22T00:00:00,1105.0,NaN,NaN,CA,201701.0,NaN,CHRY,PA,GO,1159 HUNTLEY DR,2A75,1.0,8069AA,NO STOP/STAND AM,93.0,9.999900e+04,9.999900e+04
14,1107179581,2015-12-27T00:00:00,1055.0,NaN,NaN,CA,201605.0,NaN,TOYO,PA,BK,3100 N HOLLYRIDGE DR,NaN,54.0,8058L,PREF PARKING,68.0,9.999900e+04,9.999900e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8726003,4347600423,2019-01-10T00:00:00,1414.0,NaN,NaN,CA,201808.0,NaN,MERZ,PA,BK,7650 VINELAND AVE,00355,53.0,5204A-,DISPLAY OF TABS,25.0,6.449702e+06,1.898863e+06
8726006,4347602361,2019-01-10T00:00:00,810.0,VN752,NaN,CA,NaN,NaN,FORD,PA,GY,6301 SYLMAR AV,00300,53.0,88.13B+,METER EXP.,63.0,6.426622e+06,1.889882e+06
8726010,4347602405,2019-01-10T00:00:00,1350.0,VN316,NaN,CA,201905.0,NaN,HYUN,PA,SL,14500 FRIAR ST,00300,53.0,88.13B+,METER EXP.,63.0,6.425963e+06,1.890254e+06
8726011,4347602416,2019-01-10T00:00:00,1354.0,VN914,NaN,CA,201912.0,NaN,NISS,PA,GY,14400 FRIAR ST,00300,53.0,88.13B+,METER EXP.,63.0,6.426623e+06,1.890252e+06


In [12]:
test

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,1103341116,2015-12-21T00:00:00,1251.0,NaN,NaN,CA,200304.0,NaN,NaN,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,9.999900e+04,9.999900e+04
1,1103700150,2015-12-21T00:00:00,1435.0,NaN,NaN,CA,201512.0,NaN,NaN,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,9.999900e+04,9.999900e+04
2,1104803000,2015-12-21T00:00:00,2055.0,NaN,NaN,CA,201503.0,NaN,NaN,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6.439998e+06,1.802686e+06
3,1104820732,2015-12-26T00:00:00,1515.0,NaN,NaN,CA,NaN,NaN,NaN,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6.440041e+06,1.802686e+06
5,1106226590,2015-09-15T00:00:00,19.0,NaN,NaN,CA,201507.0,NaN,NaN,VN,GY,SAN PEDRO S/O BOYD,1A35W,1.0,4000A1,NO EVIDENCE OF REG,50.0,9.999900e+04,9.999900e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8726002,4347600412,2019-01-10T00:00:00,1302.0,NaN,NaN,CA,201809.0,NaN,NaN,PA,RD,12424 MAGNOLIA BLVD,00355,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,6.439436e+06,1.882628e+06
8726004,4347602346,2019-01-10T00:00:00,805.0,VN918,NaN,CA,201909.0,NaN,NaN,PA,GY,14400 FRIAR ST,00300,53.0,88.13B+,METER EXP.,63.0,6.426623e+06,1.890252e+06
8726007,4347602372,2019-01-10T00:00:00,1234.0,VN293A,NaN,CA,201905.0,NaN,NaN,PA,GY,14501 FRIAR ST W,00300,53.0,88.13B+,METER EXP.,63.0,6.425963e+06,1.890254e+06
8726009,4347602394,2019-01-10T00:00:00,1245.0,VN686B,NaN,CA,201905.0,NaN,NaN,PA,WT,14301 DELANO ST,00300,53.0,88.13B+,METER EXP.,63.0,6.427279e+06,1.889149e+06


# Train Model

In [13]:
# Get the target value
target = train['Make']
features = train.drop('Make', axis=1)

In [14]:
# Removing Features not required for training the model
sub_features = train.drop(['Ticket number', 'Make', 'Plate Expiry Date', 'Issue Date', 'Meter Id', 'Marked Time', 'RP State Plate', 'VIN', 'Location', 'Route', 'Violation code', 'Violation Description', 'Latitude', 'Longitude'], axis=1)

In [15]:
# Data types of each feature
sub_features.dtypes

Issue time     float64
Body Style      object
Color           object
Agency         float64
Fine amount    float64
dtype: object

In [16]:
# Null values in the dataset
sub_features.isna().sum()

Issue time      863
Body Style     3030
Color          1290
Agency            6
Fine amount    2453
dtype: int64

In [17]:
# Removing all null values from the dataset
sub_features = sub_features.dropna()
sub_features.isna().sum()

Issue time     0
Body Style     0
Color          0
Agency         0
Fine amount    0
dtype: int64

In [18]:
# Convert the Object type to category for the ML algorithm
sub_features['Body Style'] = sub_features['Body Style'].astype('category')
sub_features['Color'] = sub_features['Color'].astype('category')
sub_features['Body Style Code'] = sub_features['Body Style'].cat.codes
sub_features['Color Code'] = sub_features['Color'].cat.codes

In [27]:
y = target
x = sub_features.drop(['Body Style', 'Color'], axis=1)

In [28]:
# Choose subset of data for training (Due to large dataset)
x = x[:100000]
y = y[:100000]

In [29]:
print(x.shape)
print(y.shape)

(100000, 5)
(100000,)


In [30]:
clf = DecisionTreeClassifier()
clf.fit(x, y)

DecisionTreeClassifier()

In [31]:
# Generated test sample
test_sample = [115, 54, 63, 66, 5]
clf.predict([test_sample])

array(['BMW'], dtype=object)

# Testing The Model

In [32]:
# Get the codes ofr each Color and Body Style
color = dict( zip( sub_features['Color'], sub_features['Color'].cat.codes) ) 
bd_style = dict( zip( sub_features['Body Style'], sub_features['Body Style'].cat.codes) )

In [33]:
test = test.drop(['Ticket number', 'Make', 'Plate Expiry Date', 'Issue Date', 'Meter Id', 'Marked Time', 'RP State Plate', 'VIN', 'Location', 'Route', 'Violation code', 'Violation Description', 'Latitude', 'Longitude'], axis=1)

In [34]:
test['Color'].replace(color, inplace=True)

In [35]:
test['Body Style'].replace(bd_style, inplace=True)

In [38]:
test = test.dropna()

In [39]:
clf.predict(test[0:10])

array(['NISS', 'FORD', 'NISS', 'HOND', 'TOYT', 'CADI', 'CHEV', 'HOND',
       'TOYT', 'HOND'], dtype=object)

# Save The Model

In [40]:
# save the model to disk
filename = 'finalized_model.sav'
joblib.dump(clf, filename)

['finalized_model.sav']

In [41]:
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.predict(test[0:20])
print(result)

['NISS' 'FORD' 'NISS' 'HOND' 'TOYT' 'CADI' 'CHEV' 'HOND' 'TOYT' 'HOND'
 'MITS' 'CHEV' 'MITS' 'HOND' 'HOND' 'HOND' 'KIA' 'KIA' 'GMC' 'NISS']
